In [1]:
import json
from collections import defaultdict

team_data_file = "/data/teams.json"
puzzle_data_file = "/data/puzzle-data.json"

In [2]:
teams = None
puzzles = None

with open(team_data_file, 'r') as f:
    teams = json.load(f)
    
with open(puzzle_data_file, 'r') as f:
    puzzles = json.load(f)
    
print("Loaded data for {teams} teams".format(teams=len(teams)))
print("Loaded data for {puzzles} puzzles".format(puzzles=len(puzzles)))

Loaded data for 76 teams
Loaded data for 4 puzzles


In [3]:
counts = defaultdict(lambda: {
    'team_count': 0,
    'teams_with_no_hints': 0,
    'teams_with_some_hints': 0,
    'teams_with_timeouts': 0,
    'hints_used': defaultdict(lambda: 0)
})

for team in teams:
    division = team.get('division')
    hints_used = team.get('hints_used')
    
    counts[division]['team_count'] += 1
    counts[division]['hints_used'][hints_used] += 1
    
    hint_count_key = 'teams_with_no_hints' if hints_used == 0 else 'teams_with_some_hints'
    counts[division][hint_count_key] += 1
    
    if team.get('timeouts') > 0:
        counts[division]['teams_with_timeouts'] += 1
    

for division in counts:
    div = counts[division]
    div['no_hints_percent'] = div['teams_with_no_hints'] / div['team_count']
    div['hints_percent'] = div['teams_with_some_hints'] / div['team_count']
    div['timeouts_percent'] = div['teams_with_timeouts'] / div['team_count']
    div['teams_with_no_timeouts'] = div['team_count'] - div['teams_with_timeouts']
    

In [4]:
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [5]:
# Plot team, sizes, hints used, and scores.
team_data = defaultdict(lambda: {
    'x': [],
    'y': [],
    'size': [],
})

for i, team in enumerate(teams):
    division = team.get('division')
    td = team_data[division]
    td['x'].append(team['hints_used'])
    td['y'].append(team['final_score'] / 60)
    td['size'].append(team['size']*4)
    
team_traces = []
for division, td in team_data.items():
    team_traces.append(go.Scatter(
        name = division,
        x = td['x'],
        y = td['y'],
        mode = 'markers',
        marker = dict(
            sizemode = 'diameter',
            size = td['size']
        )
    ))
    
layout = go.Layout(
    title='Team scores (bubble size = team size)',
    xaxis=dict(title='Hints Used'),
    yaxis=dict(title='Final Score (minutes)'),
    hovermode='closest',
)
fig = dict(data=team_traces, layout=layout)

iplot(fig)

In [6]:
# Pie charts for completion with and without hints by division

for division, div in counts.items():    
    hint_labels = ["No Hints", "Some Hints"]
    hint_values = [div['teams_with_no_hints'], div['teams_with_some_hints']]
    hint_domain = { 'x': [0, .48], 'y': [0,1] }
    
    timeout_labels = ["No Timeouts", "Some Timeouts"]
    timeout_values = [div['teams_with_no_timeouts'], div['teams_with_timeouts']]
    timeout_domain = { 'x': [.5, 1], 'y': [0,1] }
    
    hint_trace = go.Pie(
        labels=hint_labels,
        values=hint_values,
        domain=hint_domain
    )
    timeout_trace = go.Pie(
        labels=timeout_labels, 
        values=timeout_values, 
        domain=timeout_domain
    )
    
    layout = go.Layout(title=division)
    fig = go.Figure(data=[hint_trace, timeout_trace], layout=layout)
    iplot(fig)

In [7]:
# Bar Chart for Hints Taken by division

bar_chart_data = []

for division, div in counts.items():
    trace = go.Bar(
        x=[num for num in range(13)],
        y=[div['hints_used'][num] for num in range(13)],
        name=division
    )
    
    bar_chart_data.append(trace)
    
layout = go.Layout(
    barmode='group',
    title='Hints Taken by Division',
    xaxis=dict(title="Number of hints taken"),
    yaxis=dict(title="Number of teams")
    
)
fig = go.Figure(data=data, layout=layout)

iplot(fig)


NameError: name 'data' is not defined

In [ ]:
# Print Raw Data 
print(json.dumps(counts, indent=2))